In [ ]:
from PIL import Image
import numpy
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

In [ ]:
%run 'Map Extraction - Functions.ipynb'

In [ ]:
DATA_COMPUTED_DIRECTORY = "/media/host_data/lions/"
SIGN_DIRECTORY = "/media/host_data/lions/sign/"
REFERENCE_DIRECTORY = "/media/host_data/lions/test/sign/ref/"
TEST_DIRECTORY = "/media/host_data/lions/test/sign/"
WRITE_DIRECTORY = "/media/host_data/lions/"

In [ ]:
def signCompare(img1, img2):
    for x in range(120,132):
        for y in range(64,92):
            if(img1[y*320+x] != img2[y*320+x]):
                return False
    #for x in range(108,194):
    #    for y in range(84,91):
    #        if(img1[y*320+x] != img2[y*320+x]):
    #            return False
    return True

In [ ]:
def file_to_img_array(file):
    img = Image.open(file)
    img = img.convert('L')
    img = list(img.getdata())
    img = numpy.array(img)
    s = img.shape[0]
    img_wide = img.reshape(1,s)
    img = img_wide[0]
    return img

In [ ]:
Unavailable = file_to_img_array(REFERENCE_DIRECTORY + "Unavailable.gif")
AlsoUnavailable = file_to_img_array(REFERENCE_DIRECTORY + "AlsoUnavailable.gif")
ImageUnavailable = file_to_img_array(REFERENCE_DIRECTORY + "ImageUnavailable.gif")
SpecialEvent = file_to_img_array(REFERENCE_DIRECTORY + "SpecialEvent.gif")
comparers = {}
comparables = [0,2,5,10,15,20,25,30,40,50,60,90,120,150]

for m in comparables:
    comparers[m] = file_to_img_array(REFERENCE_DIRECTORY + str(m) + ".gif")

In [ ]:
#Tests
for t in [0, 5, 10, 15, 20, 25, 30, 40]:
    test_img = file_to_img_array(TEST_DIRECTORY + str(t) + '.gif')
    print(signCompare(test_img, comparers[t]))

In [ ]:
files = [ f for f in listdir(SIGN_DIRECTORY) if isfile(join(SIGN_DIRECTORY,f)) ]

In [ ]:
len(files)

In [ ]:
delays = np.zeros(len(files))

In [ ]:
for idx, f in enumerate(files):
    fimg = file_to_img_array(SIGN_DIRECTORY + f)
    match = False
    if not (signCompare(fimg,Unavailable) or signCompare(fimg,AlsoUnavailable)or signCompare(fimg,ImageUnavailable)or signCompare(fimg,SpecialEvent)):
        for m in comparables:
            if signCompare(fimg,comparers[m]):
                delays[idx] = m

In [ ]:
times = [get_vancouver_datetime_from_filename(f) for f in files]

In [ ]:
df = pd.DataFrame({'time':times, 'delay':delays})
df.set_index('time', inplace=True)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.to_csv(WRITE_DIRECTORY + 'sign.csv')